In [25]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import matplotlib.pyplot as plt
import data_processing

data = data_processing.load_data(download=True)
#print(data)
new_data = data_processing.convert2onehot(data)
#print(new_data)
# prepare training data
new_data = new_data.values.astype(np.float32)       # change to numpy array and float32
np.random.shuffle(new_data)
sep = int(0.7*len(new_data))

train_data = new_data[:sep]                         # training data (70%)

test_data = new_data[sep:]                          # test data (30%)

train_data_tensor = torch.from_numpy(train_data)
test_data_tensor = torch.from_numpy(test_data)

# dimensions of train data
print("DIMENSIONS OF TRAIN DATA: ", train_data_tensor.shape)
# dimensions of test data
print("DIMENSIONS OF TEST DATA: ", test_data_tensor.shape)


class GATES:
    def __init__(self):
        self.inputSize = 25 
        self.outputSize = 1
        self.hiddenSize = 1209
        
        # weights [eventually want a set of different weights for each function]
        self.Wz1 = torch.randn(self.inputSize, self.hiddenSize) # random weights for 25 x 1209 tensor. stores the parameters from the input to hidden layer;
        self.bz1 = torch.randn(1, 1) # bias is a scalar
        self.Wz2 = torch.randn(self.inputSize, self.hiddenSize) # random weights for 25 x 1209 tensor.
        self.bz2 = torch.randn(1, 1) # bias is a scalar
        self.Wx = torch.randn(self.inputSize, self.hiddenSize) # random weights for 25 x 1209 tensor
        self.bX = torch.randn(1, 1) # bias is a scalar
        self.Wy = torch.randn(self.inputSize, self.hiddenSize) # random weights for 25 x 1209 tensor
        self.bY = torch.randn(1, 1) # bias is a scalar
        
        # ANDGATE
    def andgate(self, X):
        # z1 is the first input [sum the values]
        z1 = torch.matmul(X, self.Wz1)
        z1Sum = torch.sum(z1) + self.bz1
            
        # z2 is the second input [sum the values]
        z2 = torch.matmul(X, self.Wz2)
        z2Sum = torch.sum(z2) + self.bz2
        
        # x is one possible output [sum the values]
        def x():
            x = torch.matmul(X, self.Wx)
            return torch.sum(x) + self.bX

        # y is one possible output [sum the values]
        def y():
            y = torch.matmul(X, self.Wy)
            return torch.sum(y) + self.bY

        def firstgate():
            if(z1Sum < 0):
                result = x()
            else:
                result = y()
            return result

        def secondgate():
            if(z2Sum < 0):
                result = firstgate()
            else:
                result = y()
            return result

        return secondgate()

    def notgate(self, X):

        def x():
            x = torch.matmul(X, self.Wx) + self.bX
            return torch.sum(x)

        def y():
            y = torch.matmul(X, self.Wy) + self.bY
            return torch.sum(y)

        z1 = torch.matmul(X, self.Wz1) + self.bz1
        z1Sum = torch.sum(z1)

        if(z1Sum < 0):
            result = y()

        else:
            result = x()
        
        return result

        
    def orgate(self, X):
        
        # z1 is the first input [sum the values]
        z1 = torch.matmul(X, self.Wz1)
        z1Sum = torch.sum(z1) + self.bz1
            
        # z2 is the second input [sum the values]
        z2 = torch.matmul(X, self.Wz2)
        z2Sum = torch.sum(z2) + self.bz2
        
        # x is one possible output [sum the values]
        def x():
            x = torch.matmul(X, self.Wx)
            return torch.sum(x) + self.bX

        # y is one possible output [sum the values]
        def y():
            y = torch.matmul(X, self.Wy)
            return torch.sum(y) + self.bY

        def firstgate():
            if(z1Sum < 0):
                result = x()
            else:
                result = y()
            return result

        def secondgate():
            if(z2Sum < 0):
                result = x()
            else:
                result = firstgate()
            return result

        return secondgate()
    
    def backward(self, X, y, o):
        print("Y shape is: ", y.shape)
        print("O shape is: ", o.shape) # compare scalars to one another?
        self.o_error = y - o # error in output
        print("ERROR is: " , self.o_error) # difference between output and suspected output
        #self.o_delta = self.o_error * self.sigmoidPrime(o) # multiply this error by the sigmoid
        #print("Delta is: " , self.o_delta)
        #self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        #print("Z2 error is: " , self.o_error)
        #self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        #print("Z2 delta is: " , self.z2_delta)
        #self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        #print("New W1 is: " , self.W1)
        #self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        #print("New W2 is: " , self.W2)

        
        
o = GATES()
andOutput = o.andgate(train_data_tensor)
notOutput = o.notgate(train_data_tensor)
orOutput = o.orgate(train_data_tensor)
o.backward(train_data_tensor, test_data_tensor, andOutput)

Downloaded to car.csv
DIMENSIONS OF TRAIN DATA:  torch.Size([1209, 25])
DIMENSIONS OF TEST DATA:  torch.Size([519, 25])
Y shape is:  torch.Size([519, 25])
O shape is:  torch.Size([1, 1])
ERROR is:  tensor([[28512.6426, 28513.6426, 28512.6426,  ..., 28512.6426, 28513.6426,
         28512.6426],
        [28512.6426, 28513.6426, 28512.6426,  ..., 28512.6426, 28513.6426,
         28512.6426],
        [28512.6426, 28512.6426, 28512.6426,  ..., 28512.6426, 28513.6426,
         28512.6426],
        ...,
        [28512.6426, 28512.6426, 28512.6426,  ..., 28512.6426, 28512.6426,
         28512.6426],
        [28513.6426, 28512.6426, 28512.6426,  ..., 28512.6426, 28513.6426,
         28512.6426],
        [28512.6426, 28512.6426, 28513.6426,  ..., 28512.6426, 28512.6426,
         28512.6426]])
